In [21]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from tabulate import tabulate
import re
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import os
import sys

module_path = '/home/serinatan/project/gpgpu-sim_simulations/util/job_launching/results'
if module_path not in sys.path:
    sys.path.append(module_path)
    
import helper.help_iso as hi


bench_dict = {'cut_sgemm-0':0, 'cut_sgemm-1':0, 'cut_wmma-0': 0, 'cut_wmma-1': 0, 
         'parb_stencil-0': 1, 'parb_sgemm-0': 0,
         'parb_lbm-0': 1, 'parb_spmv-0': 1, 'parb_cutcp-0': 0}

mpl.style.use('seaborn-paper')





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read CSVs

In [44]:
df_seq = pd.read_csv('seq.csv')
df_intra = pd.read_csv('intra.csv')
df_inter = pd.read_csv('inter.csv')
print(df_inter.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
       'avg_mrq_latency', 'stall_core_ldst', 'stall_icnt_to_l2', 'l2_BW',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'stall_l2_to_icnt',
       'l1D_miss_rate', 'l2_miss_rate', 'l2_total_accesses', 'packet_lat_out',
       'network_lat_out', 'inject_out', 'accepted_out', 'packet_lat_in',
       'network_lat_in', 'inject_in', 'accepted_in',
       'mem_subpartition_parallism', 'mem_subpartition_parallism_util',
       'L2_reservation_fail', 'dram_eff', 'dram_bw', 'row_buffer_locality',
       'mrqq', 'total_cmd', 'wasted_col', 'wasted_row', 'mem_idle', 'CCDLc',
       'WTRc', 'RTWc', 'RCDc', 'RCDWRc'],
      dtype='object')


## Process columns

In [45]:
# scale all IPC to baseline
baseline_dict = pd.Series(df_seq.ipc.values,index=df_seq.pair_str).to_dict() 

def norm_ipc_row(row):
    return row['ipc'] / baseline_dict[row['pair_str']]

df_intra['norm_ipc'] = df_intra.apply(lambda row: norm_ipc_row(row), axis=1)
df_inter['norm_ipc'] = df_inter.apply(lambda row: norm_ipc_row(row), axis=1)

df_intra['avg_dram_bw'] = df_intra['dram_bw'].transform(hi.avg_array)
df_inter['avg_dram_bw'] = df_inter['dram_bw'].transform(hi.avg_array)

df_intra['dram_idleness'] = np.divide(df_intra['mem_idle'].transform(hi.avg_array),
                                      df_intra['total_cmd'].transform(hi.avg_array))
df_inter['dram_idleness'] = np.divide(df_inter['mem_idle'].transform(hi.avg_array),
                                      df_inter['total_cmd'].transform(hi.avg_array))

# decompose gpgpu-sim configs
hi.process_config_column('intra', 'l2', df=df_intra)
hi.process_config_column('inter', 'l2', df=df_inter)

print(df_intra.columns)


Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
       'avg_mrq_latency', 'stall_core_ldst', 'stall_icnt_to_l2', 'l2_BW',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'stall_l2_to_icnt',
       'l1D_miss_rate', 'l2_miss_rate', 'l2_total_accesses', 'packet_lat_out',
       'network_lat_out', 'inject_out', 'accepted_out', 'packet_lat_in',
       'network_lat_in', 'inject_in', 'accepted_in',
       'mem_subpartition_parallism', 'mem_subpartition_parallism_util',
       'L2_reservation_fail', 'dram_eff', 'dram_bw', 'row_buffer_locality',
       'mrqq', 'total_cmd', 'wasted_col', 'wasted_row', 'mem_idle', 'CCDLc',
       'WTRc', 'RTWc', 'RCDc', 'RCDWRc', 'norm_ipc', 'avg_dram_bw',
       'dram_idleness', 'intra', 'l2'],
      dtype='object')


In [46]:
def print_intra(df, benchmark):
    filename = '{0}-{1}.pdf'.format(benchmark, 'intra')
    filename = os.path.join('plots', filename)
    with PdfPages(filename) as pdf:
        hi.plot_page_intra(df, 'norm_ipc', benchmark, pdf)
        hi.plot_page_intra(df, 'avg_dram_bw', benchmark, pdf)
        hi.plot_page_intra(df, 'dram_idleness', benchmark, pdf)
        hi.plot_page_intra(df, 'l2_miss_rate', benchmark, pdf)
        hi.plot_page_intra(df, 'l2_BW', benchmark, pdf)
        hi.plot_page_intra(df, 'l2_total_accesses', benchmark, pdf)
        hi.plot_page_intra(df, 'l1D_miss_rate', benchmark, pdf)
        hi.plot_page_intra(df, 'avg_mem_lat', benchmark, pdf)

def print_intra_inter(df_intra, df_inter, benchmark):
    filename = '{0}-{1}.pdf'.format(benchmark, 'both')
    filename = os.path.join('plots', filename)
    with PdfPages(filename) as pdf:
        hi.plot_page_intra_inter(df_intra, df_inter, 'norm_ipc', benchmark, pdf)
 

In [19]:
print_intra_inter(df_intra, df_inter, 'parb_stencil-0')

In [20]:
print_intra_inter(df_intra, df_inter, 'parb_cutcp-0')

## Intra SM Only

In [42]:
bench_list = list(df_intra['pair_str'].unique())
bench_list.sort()

In [35]:
for bench in bench_list:
    print(bench)
    print_intra(df_intra, bench)

cut_sgemm-1
cut_wmma-1
parb_stencil-0
parb_spmv-0
cut_sgemm-0
parb_cutcp-0
parb_lbm-0
cut_wmma-0


In [51]:
fig_tot, axs = plt.subplots(2, 4, figsize=(40, 30))   
axs = axs.flat

for ax, bench in zip(axs, bench_list):
    _df = df_intra[df_intra['pair_str'] == bench]
    
    hi.plot_heatmap(_df, x_key='intra', y_key='l2', z_key='norm_ipc', title=bench, axis=ax, scale=1.2)


fig_tot.suptitle('Intra, Normalized IPC', fontsize=18)
fig_tot.savefig('plots/total.pdf')
plt.close()


